In [51]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from config import geoapify_key

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

import warnings
warnings.filterwarnings("ignore")

In [52]:
# Import the WeatherPy_vacation.csv file. 
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,0,byron bay,AU,79.95,broken clouds,-28.6500,153.6167,Lord Byron Resort
1,1,punta arenas,CL,53.11,clear sky,-53.1500,-70.9167,Hotel Lacolet
2,2,faanui,PF,81.16,overcast clouds,-16.4833,-151.7500,Oa Oa Lodge
3,3,new norfolk,AU,72.72,scattered clouds,-42.7826,147.0587,Woodbridge on the Derwent
4,4,cayenne,GF,73.44,broken clouds,4.9333,-52.3333,Central Hôtel
...,...,...,...,...,...,...,...,...
368,710,oussouye,SN,72.12,broken clouds,12.4850,-16.5469,Campement Alulum
369,713,sur,OM,67.55,scattered clouds,22.5667,59.5289,Sur Hotel
370,714,altamont,US,38.89,clear sky,42.2068,-121.7372,Quality Inn
371,716,dubbo,AU,80.73,scattered clouds,-32.2500,148.6167,Blue Gum Motel


In [53]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Weather", "Country", "Hotel Name"])
# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Weather,Country,Hotel Name)

In [54]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "cape town"]
vacation_end = vacation_df.loc[vacation_df["City"] == "cape town"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "richards bay"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "plettenberg bay"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "hermanus"]

In [55]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
12,19,cape town,ZA,75.90,scattered clouds,-33.9258,18.4232,Townhouse Hotel
22,36,richards bay,ZA,72.36,broken clouds,-28.7830,32.0377,Protea Hotel
58,102,plettenberg bay,ZA,69.48,overcast clouds,-34.0527,23.3716,Milkwood Manor
9,14,hermanus,ZA,73.80,broken clouds,-34.4187,19.2345,Aloe guest house
12,19,cape town,ZA,75.90,scattered clouds,-33.9258,18.4232,Townhouse Hotel


In [56]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,Lng,Lat
12,18.4232,-33.9258
22,32.0377,-28.7830
58,23.3716,-34.0527
9,19.2345,-34.4187
12,18.4232,-33.9258


In [57]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Weather", "Country", "Hotel Name"])


In [58]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Weather,Country,Hotel Name)

In [59]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [60]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'd9b67293020d4c2eab0d4a32c5e35d58',
 'waypoints': '-33.9258,18.4232|-28.783,32.0377|-34.0527,23.3716|-34.4187,19.2345|-33.9258,18.4232'}

In [61]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params).json()

# Convert the API response to JSON format
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [18.4232, -33.9258], 'original_index': 0},
     {'location': [32.0377, -28.783], 'original_index': 1},
     {'location': [23.3716, -34.0527], 'original_index': 2},
     {'location': [19.2345, -34.4187], 'original_index': 3},
     {'location': [18.4232, -33.9258], 'original_index': 4}],
    'units': 'metric',
    'distance': 3699513,
    'distance_units': 'meters',
    'time': 130998.661,
    'legs': [{'distance': 1806976,
      'time': 59122.394,
      'steps': [{'from_index': 0,
        'to_index': 3,
        'distance': 158,
        'time': 9.492,
        'instruction': {'text': 'Drive southeast on Caledon Street.'}},
       {'from_index': 3,
        'to_index': 24,
        'distance': 390,
        'time': 32.097,
        'instruction': {'text': 'Turn left onto Buitenkant Street/M59. Continue on M59.'}},
       {'from_index': 24,
        'to_index': 77,
        'distance': 2096,
      

In [62]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

In [63]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
# YOUR CODE HERE (double for loop) - and then two apend lines into a list

for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

In [64]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()
# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["Latitude"] = latitude
# Display sample data
route_df

,longitude,Latitude
0,18.422919,-33.926062
1,18.423419,-33.926432
2,18.424143,-33.926975
3,18.424191,-33.927011
4,18.424246,-33.926958
...,...,...
45877,18.422668,-33.925761
45878,18.422662,-33.925805
45879,18.422668,-33.925848
45880,18.422685,-33.925889


In [65]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(
    height = 500,
    width = 700,
    line_width = 3,
    color = "red")

In [66]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Weather,Country,Hotel Name)
   .Path.I   :Path   [longitude,Latitude]